# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fcc5c1d9790>
├── 'a' --> tensor([[ 0.6640,  0.9327,  0.2144],
│                   [ 1.1268,  0.2577, -0.2982]])
└── 'x' --> <FastTreeValue 0x7fcc5c1d9820>
    └── 'c' --> tensor([[ 0.8634,  0.4731, -0.9239, -0.3876],
                        [-1.2594, -1.0365, -1.8159, -0.1455],
                        [ 2.0740, -0.0498,  1.8884,  1.4686]])

In [4]:
t.a

tensor([[ 0.6640,  0.9327,  0.2144],
        [ 1.1268,  0.2577, -0.2982]])

In [5]:
%timeit t.a

64.6 ns ± 0.0333 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fcc5c1d9790>
├── 'a' --> tensor([[ 1.6027, -1.1830,  1.1468],
│                   [ 0.8017,  0.4256,  1.4531]])
└── 'x' --> <FastTreeValue 0x7fcc5c1d9820>
    └── 'c' --> tensor([[ 0.8634,  0.4731, -0.9239, -0.3876],
                        [-1.2594, -1.0365, -1.8159, -0.1455],
                        [ 2.0740, -0.0498,  1.8884,  1.4686]])

In [7]:
%timeit t.a = new_value

64.4 ns ± 0.0482 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.6640,  0.9327,  0.2144],
               [ 1.1268,  0.2577, -0.2982]]),
    x: Batch(
           c: tensor([[ 0.8634,  0.4731, -0.9239, -0.3876],
                      [-1.2594, -1.0365, -1.8159, -0.1455],
                      [ 2.0740, -0.0498,  1.8884,  1.4686]]),
       ),
)

In [10]:
b.a

tensor([[ 0.6640,  0.9327,  0.2144],
        [ 1.1268,  0.2577, -0.2982]])

In [11]:
%timeit b.a

51.1 ns ± 0.0371 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7785,  1.0169, -1.0686],
               [-0.8001, -0.3069,  0.2876]]),
    x: Batch(
           c: tensor([[ 0.8634,  0.4731, -0.9239, -0.3876],
                      [-1.2594, -1.0365, -1.8159, -0.1455],
                      [ 2.0740, -0.0498,  1.8884,  1.4686]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 1.19 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

860 ns ± 0.292 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 175 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 534 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 2.79 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fcbafeaf5e0>
├── 'a' --> tensor([[[ 0.6640,  0.9327,  0.2144],
│                    [ 1.1268,  0.2577, -0.2982]],
│           
│                   [[ 0.6640,  0.9327,  0.2144],
│                    [ 1.1268,  0.2577, -0.2982]],
│           
│                   [[ 0.6640,  0.9327,  0.2144],
│                    [ 1.1268,  0.2577, -0.2982]],
│           
│                   [[ 0.6640,  0.9327,  0.2144],
│                    [ 1.1268,  0.2577, -0.2982]],
│           
│                   [[ 0.6640,  0.9327,  0.2144],
│                    [ 1.1268,  0.2577, -0.2982]],
│           
│                   [[ 0.6640,  0.9327,  0.2144],
│                    [ 1.1268,  0.2577, -0.2982]],
│           
│                   [[ 0.6640,  0.9327,  0.2144],
│                    [ 1.1268,  0.2577, -0.2982]],
│           
│                   [[ 0.6640,  0.9327,  0.2144],
│                    [ 1.1268,  0.2577, -0.2982]]])
└── 'x' --> <FastTreeValue 0x7fcbafeaf1f0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.1 µs ± 74.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fcbafeafd00>
├── 'a' --> tensor([[ 0.6640,  0.9327,  0.2144],
│                   [ 1.1268,  0.2577, -0.2982],
│                   [ 0.6640,  0.9327,  0.2144],
│                   [ 1.1268,  0.2577, -0.2982],
│                   [ 0.6640,  0.9327,  0.2144],
│                   [ 1.1268,  0.2577, -0.2982],
│                   [ 0.6640,  0.9327,  0.2144],
│                   [ 1.1268,  0.2577, -0.2982],
│                   [ 0.6640,  0.9327,  0.2144],
│                   [ 1.1268,  0.2577, -0.2982],
│                   [ 0.6640,  0.9327,  0.2144],
│                   [ 1.1268,  0.2577, -0.2982],
│                   [ 0.6640,  0.9327,  0.2144],
│                   [ 1.1268,  0.2577, -0.2982],
│                   [ 0.6640,  0.9327,  0.2144],
│                   [ 1.1268,  0.2577, -0.2982]])
└── 'x' --> <FastTreeValue 0x7fcbafeaf760>
    └── 'c' --> tensor([[ 0.8634,  0.4731, -0.9239, -0.3876],
                        [-1.2594, -1.0365, -1.8159, -0.1455],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 45.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.7 µs ± 135 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.6640,  0.9327,  0.2144],
                [ 1.1268,  0.2577, -0.2982]],
       
               [[ 0.6640,  0.9327,  0.2144],
                [ 1.1268,  0.2577, -0.2982]],
       
               [[ 0.6640,  0.9327,  0.2144],
                [ 1.1268,  0.2577, -0.2982]],
       
               [[ 0.6640,  0.9327,  0.2144],
                [ 1.1268,  0.2577, -0.2982]],
       
               [[ 0.6640,  0.9327,  0.2144],
                [ 1.1268,  0.2577, -0.2982]],
       
               [[ 0.6640,  0.9327,  0.2144],
                [ 1.1268,  0.2577, -0.2982]],
       
               [[ 0.6640,  0.9327,  0.2144],
                [ 1.1268,  0.2577, -0.2982]],
       
               [[ 0.6640,  0.9327,  0.2144],
                [ 1.1268,  0.2577, -0.2982]]]),
    x: Batch(
           c: tensor([[[ 0.8634,  0.4731, -0.9239, -0.3876],
                       [-1.2594, -1.0365, -1.8159, -0.1455],
                       [ 2.0740, -0.0498,  1.8884,  1.4686]],
         

In [26]:
%timeit Batch.stack(batches)

79.2 µs ± 136 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.6640,  0.9327,  0.2144],
               [ 1.1268,  0.2577, -0.2982],
               [ 0.6640,  0.9327,  0.2144],
               [ 1.1268,  0.2577, -0.2982],
               [ 0.6640,  0.9327,  0.2144],
               [ 1.1268,  0.2577, -0.2982],
               [ 0.6640,  0.9327,  0.2144],
               [ 1.1268,  0.2577, -0.2982],
               [ 0.6640,  0.9327,  0.2144],
               [ 1.1268,  0.2577, -0.2982],
               [ 0.6640,  0.9327,  0.2144],
               [ 1.1268,  0.2577, -0.2982],
               [ 0.6640,  0.9327,  0.2144],
               [ 1.1268,  0.2577, -0.2982],
               [ 0.6640,  0.9327,  0.2144],
               [ 1.1268,  0.2577, -0.2982]]),
    x: Batch(
           c: tensor([[ 0.8634,  0.4731, -0.9239, -0.3876],
                      [-1.2594, -1.0365, -1.8159, -0.1455],
                      [ 2.0740, -0.0498,  1.8884,  1.4686],
                      [ 0.8634,  0.4731, -0.9239, -0.3876],
                      [-1.2594, -

In [28]:
%timeit Batch.cat(batches)

142 µs ± 238 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 1.36 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
